In [1]:
import json
from pathlib import Path
import h5py
from datasets import Dataset, DatasetDict
import os
#from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq


/Users/ahmed/.pyenv/versions/3.13.5/envs/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ---------- setup ----------
root_path = Path(os.getcwd()).parent

In [3]:
root_path

PosixPath('/Users/ahmed/PycharmProjects/graph-reasoning-project')

In [ ]:
SPECIAL_TOKENS = ["<triplet>", "<subj>", "<obj>"]

def _decode(b):
    return b.decode("utf-8", "ignore") if isinstance(b, (bytes, bytearray)) else b

def _pick_text(d):
    # Moritz’s JSON rows usually have 'text'. Fall back to common alternatives.
    for k in ("text", "context", "sentence", "source"):
        if k in d and d[k]:
            return d[k]
    return ""

def _pick_triplets(d):
    # Usually 'triplets'. Be tolerant to variants.
    for k in ("triplets", "relations", "labels", "targets", "target"):
        if k in d and d[k] is not None:
            return d[k]
    return []

def linearize_triplets(triplets):
    parts = []
    for t in triplets or []:
        if isinstance(t, dict):
            s = str(t.get("subject","")).strip()
            r = str(t.get("relation","")).strip()
            o = str(t.get("object","")).strip()
        elif isinstance(t, (list, tuple)) and len(t) >= 3:
            s, r, o = map(lambda x: str(x).strip(), t[:3])
        else:
            continue
        if s and r and o:
            parts += ["<triplet>", s, "<subj>", r, "<obj>", o]
    return " ".join(parts)


In [ ]:
def load_rebel_hdf5(path):
    """
    Loads an HDF5 created by Moritz's script:
      - datasets at the root named by split ('train', 'val', 'test', etc.)
      - each element is a JSON string with at least 'text' and 'triplets' (or variants)
    Returns a DatasetDict with splits as found in the file.
    """
    dd = {}
    with h5py.File(Path(path), "r") as f:
        # auto-detect split names that are datasets (not groups)
        split_names = [k for k in f.keys() if isinstance(f[k], h5py.Dataset)]
        if not split_names:
            raise RuntimeError("No split datasets found in HDF5 (expected datasets like 'train', 'val', 'test').")

        # normalize aliases (val/dev -> validation)
        alias = {"val":"validation", "valid":"validation", "dev":"validation"}
        for split in split_names:
            canon = alias.get(split.lower(), split.lower())
            arr = f[split][()]  # array of bytes/strings
            rows = [json.loads(_decode(x)) for x in arr]

            contexts = [_pick_text(d) for d in rows]
            triplets = [_pick_triplets(d) for d in rows]

            dd[canon] = Dataset.from_dict({"context": contexts, "triplets": triplets})

    return DatasetDict(dd)

# ---------- use it ----------
ds = load_rebel_hdf5(os.path.join(root_path, "baselines","GraphLanguageModels","data", "rebel_dataset", "rebel.hdf5"))

print("Splits found:", list(ds.keys()))
some_split = list(ds.keys())[0]
print(ds[some_split][0])  # peek first example

# Build (source, target) for seq2seq
def build_io(example):
    return {"source": example["context"], "target": linearize_triplets(example.get("triplets", []))}

ds_io = ds.map(build_io, remove_columns=[c for c in ds[next(iter(ds))].column_names if c not in ("context","triplets")])

# Optional: drop empty targets
ds_io = ds_io.filter(lambda e: bool(e["target"].strip()))
